In [1]:
import tkinter
from tkinter import filedialog
import os



def search_for_file_path ():
    root = tkinter.Tk()
    root.withdraw() #use to hide tkinter window
    currdir = os.getcwd()
    tempfile = filedialog.askopenfilename(parent=root, initialdir=currdir, title='Please select a file')
    if len(tempfile) > 0:
        print ("You chose: %s" % tempfile)
        
    return tempfile

def folder_path_for_corrected_files ():
    root = tkinter.Tk()
    root.withdraw() #use to hide tkinter window
    currdir = os.getcwd()   
    tempdir = filedialog.askdirectory(parent=root, initialdir=currdir, title='Please select a directory')
    if len(tempdir) > 0:
        print ("You chose: %s" % tempdir)    
        
    return tempdir

def get_grad_date():
    for x in list_of_records:
        for y in x:
            if y[0] == 'DTP' and y[1] == '007':
                grad_date = y[3]

                return grad_date
        
############
#filetoread = search_for_file_path ()
#write_to_new_file = folder_path_for_corrected_files()
#grad_date = input('enter grad date: ')
#filetoread = 'sfrnslc_1853615.txt'
#grad_date = get_grad_date() #'20220525'
###########


def sepr8_file_sections(filetoread):
    file_headers = []
    file_records = []
    file_footer = []

    with open(filetoread, "r") as f:
        for x in f:
            if x.startswith('ISA|') or x.startswith('GS|'):
                x = x.strip('\n')
                file_headers.append(x)
            elif x.startswith('GE|') or x.startswith('IEA|'):
                x = x.strip('\n')
                file_footer.append(x)
            else:
                x = x.strip('\n')
                file_records.append(x)
                
    return file_headers,file_records,file_footer

##############
#file_headers,file_records,file_footer = sepr8_file_sections(filetoread)
##############


def parse_student_records(file_records):
    record_rows_start = []
    record_rows_end = []
    
    for i,x in enumerate(file_records):

        if x.startswith('ST|'):
            record_rows_start.append(i)
        elif x.startswith('SE|'):
            record_rows_end.append((i+1))

    group_records = zip(record_rows_start,record_rows_end)

    list_of_records = []

    record_number = 0
    for start,end in group_records:
        record_number += 1
        record_set = []
        for i,x in enumerate(file_records[start:end]):
            x = x.split('|')
            record_set.append(x)
        list_of_records.append(record_set)
        
    return list_of_records

###############
#list_of_records = parse_student_records(file_records)
###############


def generate_list_problem_records():
    student_major_info = []
    
    for i,student_record in enumerate(list_of_records):
        student_fos2 = []
        if student_record[2][1] != 'EB4':
            for x in student_record:
                if x[0] == 'ENT' and x[1] == '03':
                    student_id = x[4]
                if x[0] == 'FOS' and x[1] == 'M':
                    student_fos_cipc = x[3]
                    student_fos_major = x[4]
                if x[0] == 'FOS' and x[1] == 'P' and x[3] == student_fos_cipc:
                    student_fos2.append(x[6])
                        
            student_major_info.append([i,student_record[2][1],student_id,student_fos_cipc,student_fos_major,student_fos2[0]])
        
    return student_major_info

def write_failed_record_logs(content_to_write):
    file_path = f'{write_new_file_to}/failed_records.txt'
    with open(file_path, "w") as f:
        first_line = f"These records required intervention, number of records: {len(content_to_write)}"
        print(first_line,file = f)
        for x in content_to_write:
            print(x,file = f)
            
#####################
#problem_records = generate_list_problem_records()
#write_failed_record_logs(problem_records)
#print(problem_records)
#for x in problem_records:
#    print(x)
#####################    

    
def fix_student_record(list_of_records,record_number,grad_date):
    x = list_of_records[record_number]
    
    #correct ENR line update to EB4, clear expected grad date, update grad date
    if x[2][0] == 'ENR' and x[2][1] != 'EB4':
        x[2][1] = 'EB4'
        x[2][3] = ''
        x[2][4] = ''
        #x[2][13] = grad_date
    
    #add DTP row
    if x[4][0] != 'DTP':      
        list_of_records[record_number].insert(4,['DTP','007','D8',grad_date]) #used date variable here
    
    #correct second ENR line
    for line in x:
        if line[0] == 'ENR' and line[1] != 'EB4' and line[1] != 'EB3':
            line[1] = 'EB4'
            line[13] = grad_date
    
    #correct row count
    row_count = len(x)
    if x[(row_count-1)][0] == 'SE':
        x[(row_count-1)][1] = str(row_count)
    else:
        print('error updating row count for record number',record_number)
        
#############
# Need a for loop to fix all problem records need switch to decide when to delete
#fix_student_record(list_of_records,902,grad_date)
#############


def fix_line_count_record_num():
    for i,x in enumerate(list_of_records):
        record_number_formatted = str((i+1)).zfill(9)
        record_line_count = len(x)
        record_last_line = (len(x)-1)

        x[0][2] = str(record_number_formatted)
        x[1][2] = str(record_number_formatted)
        x[(len(x)-1)][1] = str(record_line_count)
        x[(len(x)-1)][2] = str(record_number_formatted)
        
#########        
#fix_line_count_record_num()
###########


def parse_footer_file():
    parsed_footer = []
    for i,x in enumerate(file_footer):
        parsed_footer.append(x.split('|'))

    return parsed_footer

########        
#parsed_footer = parse_footer_file()
########


def update_footer_line_count():
    for i,x in enumerate(parsed_footer):
        if x[0] == 'GE':
            x[1] = str(len(list_of_records))
            

def write_submit_file(file_headers,list_of_records,file_footer):
    file_path = f'{write_new_file_to}/00111300.clr'
    with open(file_path, "w") as f:
        for x in file_headers:
            print(x,file = f)

        for x in list_of_records:
            for y in x:
                print('|'.join(y),file = f)

        for x in file_footer:
            print('|'.join(x),file = f)
            
#########
#write_submit_file(file_headers,list_of_records,file_footer)
#########

In [2]:
# obtain user variables
filetoread = search_for_file_path ()
write_new_file_to = folder_path_for_corrected_files()

You chose: C:/Users/rwalden/PyOracle/NCH_grad_file_checker/sfrnslc_1853615.txt
You chose: C:/Users/rwalden/PyOracle/NCH_grad_file_checker


In [3]:
# read file break into header records and footer
file_headers,file_records,file_footer = sepr8_file_sections(filetoread)
# parse the records for processing
list_of_records = parse_student_records(file_records)
# get grad date from file
grad_date = get_grad_date()
#grad_date = input('enter grad date: ')

In [4]:
#generate a list of records with an error
problem_records = generate_list_problem_records()
#write records to file for review
write_failed_record_logs(problem_records)
#for x in problem_records:
    #print(x)

In [5]:
#fix the records
problem_records = generate_list_problem_records()
for x in problem_records:
    fix_student_record(list_of_records,x[0],grad_date)
    

In [6]:
#fix the record line counts
fix_line_count_record_num()

In [7]:
#fix the footer report line count
parsed_footer = parse_footer_file()
update_footer_line_count()

In [8]:
#write the corrected file to disk
write_submit_file(file_headers,list_of_records,parsed_footer)

In [ ]:
#list_of_records[489]

In [4]:
def generate_list_problem_records():
    #problem_records = []
    student_major_info = []
    
    for i,student_record in enumerate(list_of_records):
        student_fos2 = []
        if student_record[2][1] != 'EB4':
            for x in student_record:
                if x[0] == 'ENT' and x[1] == '03':
                    student_id = x[4]
                if x[0] == 'FOS' and x[1] == 'M':
                    student_fos_cipc = x[3]
                    student_fos_major = x[4]
                if x[0] == 'FOS' and x[1] == 'P' and x[3] == student_fos_cipc:
                    student_fos2.append(x[6])
                        
            student_major_info.append([i,student_record[2][1],student_id,student_fos_cipc,student_fos_major,student_fos2[0]])
        
    return student_major_info

In [11]:
def write_failed_record_logs(content_to_write):
    file_path = f'{write_new_file_to}/failed_records.txt'
    with open(file_path, "w") as f:
        first_line = f"These records required intervention, number of records: {len(content_to_write)}"
        print(first_line,file = f)
        for x in content_to_write:
            print(x,file = f)

In [41]:
#working towards deleting from problem list those which will be removed from report
def find_records_to_del(problem_records):
    problem_records = generate_list_problem_records()
    list_to_delete = ['8','11']
    for i,x in enumerate(problem_records):
        #print(x)
        if x[0] in list_to_delete:
            print(x[0])
            print(i,problem_records[x])
            #del problem_records[x]
            
    return problem_records

In [42]:
problem_records = generate_list_problem_records()

#write_failed_record_logs(problem_records)


print(len(problem_records))
problem_records = find_records_to_del(problem_records)
#del problem_records[901]
print(len(problem_records))
#for x in problem_records:
    #print(x)

323
323


In [19]:


def get_grad_date():
    for x in list_of_records:
        for y in x:
            if y[0] == 'DTP' and y[1] == '007':
                grad_date = y[3]

                return grad_date
        
    
    
get_grad_date()

'20220507'